# **Problem 4**

***Note:*** You are only allowed to use pandas and numpy libraries if needed.

## **Part a):**

Complete the following function to implement a dynamic programming algorithm that finds the optimal global alignment of two input DNA sequences.

---
**Inputs:**

1- path_fasta_1 (string): path to "Fasta" format file of first sequence

2- path_fasta_2 (string): path to "Fasta" format file of second sequence

3- path_score_mat (string): path to substitution score matrix.

4- gap_penalty (negative float): a enagtive real number. 

---

**Outputs:** your function should return 3 values:

1- The optimal global alignment score

2- a string representing the alignment of first input sequence (any one, if more than one optimal solutions exist; use "-" for indels)

3- a string representing the alignment of second input sequence (any one, if more than one optimal solutions exist; use "-" for indels)


**Output example:** 5, "ACCC--AC", "AC-CGGAC"

---

**Note** since you will be implementing a dynamic programming alignment algorithm, we expect your program to be of **quadratic time** complexity (i.e., two sequences of length $n$ can be aligned in O($n^2$) time).

In [89]:
import numpy as np
def optimal_global_alignment(path_fasta_1, path_fasta_2, path_score_mat, gap_penalty = -200):
    # Read two sequences and substitution score matrix 
    fasta_1 = open(path_fasta_1)
    fasta_2 = open(path_fasta_2)
    path_score = open(path_score_mat)
    
    file = path_score.readlines()
    matrix_list = list()
    for line in file:
        line = line.split()
        matrix_list.append(line)

    matrix_dict = {}
    for i in range(1, len(matrix_list)):
        matrix_dict[matrix_list[i][0]] = {}
        for j in range(len(matrix_list[0])):
            matrix_dict[matrix_list[i][0]][matrix_list[0][j]] = matrix_list[i][j + 1]
    
    for line in fasta_1:
        line1 = line.strip()
    for line in fasta_2:
        line2 = line.strip()
        
    # Create DP matrix
    matrix = np.zeros((len(line1)+1, len(line2)+1))
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            if i == 0:
                matrix[i][j] = j * gap_penalty
            elif j == 0:
                matrix[i][j] = i * gap_penalty
            else:
                temp1 = matrix[i-1][j-1] + int(matrix_dict[line1[i-1]][line2[j-1]])
                temp2 = matrix[i-1][j] + gap_penalty
                temp3 = matrix[i][j-1] + gap_penalty
                matrix[i][j] = max(temp1, temp2, temp3)
#     print(matrix)
    aligned_1 = ""
    aligned_2 = ""
    i = matrix.shape[0] - 1
    j = matrix.shape[1] - 1
    score = matrix[i][j]
    while (i > 0 and j > 0):
        if (i >= 0 and j >= 0 and matrix[i][j] == matrix[i-1][j-1] + int(matrix_dict[line1[i-1]][line2[j-1]])):
            aligned_1 += line1[i-1]
            aligned_2 += line2[j-1]
            i -= 1
            j -= 1
        elif (i >= 0 and j >= 0 and matrix[i][j] == matrix[i-1][j] + gap_penalty):
            aligned_1 += line1[i-1]
            aligned_2 += "-"
            i -= 1
        else:
            aligned_1 += "-"
            aligned_2 += line2[j-1]
            j -= 1

#     print(aligned_1[::-1])
#     print(aligned_2[::-1])
    aligned_1 = aligned_1[::-1]
    aligned_2 = aligned_2[::-1]
    return score, aligned_1, aligned_2

In [90]:
path_fasta_1 = "./prob4_data/data_example/seq1.fasta"
path_fasta_2 = "./prob4_data/data_example/seq2.fasta"
path_score_mat = "./prob4_data/data_example/substitution_matrix.txt"

optimal_global_alignment(path_fasta_1, path_fasta_2, path_score_mat)

(3362.0,
 'CTCGTCCTAAAAAAAA-GGAACGCGTGACGCGCTAAAAG-CATTCGTGCC',
 'GTCGTCCTAAAAAAAAGGGAACGCGTGATGCGCTAAAAGACATCAGTGCC')

## **Part b):**

Run your alignment code created in the last part on the 10 provided pair of sequences (data_1 to data_10). For each alignment, use the provided substitution matrix (same for all 10 alignments) and gap penalty of -200. For each alignment $i$ ($i \in \{1..10\}$) write the results in a text file named "align_data_i.txt" in the following format:

"The optimal alignment score between given sequences has score X"

"aligned sequence 1"

"aligned sequence 2"

---

**Example:** for data_example, the output (align_data_example.txt) should be:

The optimal alignment score between given sequences has score 3362.0

CTCGTCCTAAAAAAAA-GGAACGCGTGACGCGCTAAAAG-CATTCGTGCC

GTCGTCCTAAAAAAAAGGGAACGCGTGATGCGCTAAAAGACATCAGTGCC

In [102]:
for i in range(1,11):
    path = "./prob4_data/data_" + str(i)
    path_fasta_1 = path + "/seq1.fasta"
    path_fasta_2 = path + "/seq2.fasta"
    path_score_mat = path + "/substitution_matrix.txt"
    score, aligned_1, aligned_2 = optimal_global_alignment(path_fasta_1, path_fasta_2, path_score_mat)
    output = "The optimal alignment score between given sequences has score " + str(score) + '\n'
    output += aligned_1 + '\n'
    output += aligned_2 + '\n'
    print(output)
    output_file = './align_data_' + str(i) + '.txt'
    with open(output_file, 'w') as f:
        f.write(output)

The optimal alignment score between given sequences has score 2809.0
TTGTAGGCTCCCGCA-GTGCTGGTCAAAACGAGCTCCTCGGTATAAAAAC
TTATAGGCTTCGGCAGGTG-TGGTC-AAACGA-CTACTCGGTATAAAAAC

The optimal alignment score between given sequences has score 2974.0
TTAGTACAAAGTATAGTCACCTCAGAAGCTTGTTGGCTCAATCCGAGT
TTAATA-AATGTATAGTCACCTCACGA-CTTGTTGCCTCAATTCGAGT

The optimal alignment score between given sequences has score 6003.0
GGCACAGTATC-GTGCTTACCCTGCTTCGTCCGTTCGCTCTAGTAGATCAGCTTTGCTTAAACCTGACTGAGCAGTAAGAGGTGATAGTGTCGGGGCG
GG-ACAGAATCGGTGCTTGCCCT-CTTCGTTCGTTCG-TCTAGTAGACCATCTTTCCTAAAACCCGACTG-GCAGTAAGAGGTGATAGTGTCAGGGCG

The optimal alignment score between given sequences has score 5615.0
GGAAACAATACCTCTGCCCATATGCGGCAATGTCTATTTAATTGGCGTATCTTTCAGTCGTGGTTCGTGCTTTACTCCGATTATTCTCTGAGTATGG
GGCAACAATATCTCCGCCCA-CTGCGGCGATGTCTA--TAATTGGCGTATCTTTCTGCCGTGATTCGTGATTTACTACCATTATACTCTGAGTA-GG

The optimal alignment score between given sequences has score 16871.0
CGG-GGAG-CACGACAGAGGGACAAGTCC-GAATTATTTCTCGCGAGGAACCACGC